In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

In [15]:
from get_rainfall_data import load_rainfall_data_from_chirps

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import rasterio as rio
import rioxarray
from shapely.geometry import shape
import seaborn as sns
import geopandas as gpd

In [2]:
from s2cloudless import S2PixelCloudDetector
from odc.algo import to_rgba
from odc.stac import stac_load, configure_rio
import dask
from deafrica_tools.plotting import rgb, display_map
from pystac_client import Client

-  Digital Earth Africa data is stored on S3 in Cape Town, Africa. 
- To load the data, we must configure rasterio with the appropriate AWS S3 endpoint.

In [4]:
os.environ["AWS_S3_ENDPOINT"] = "s3.af-south-1.amazonaws.com"
os.environ["AWS_NO_SIGN_REQUEST"] = "true"

- The configuration below must be used when loading any Digital Earth Africa data through the STAC API.

In [8]:
client = dask.distributed.Client()

configure_rio(
    cloud_defaults = True,
    aws = {"aws_unsigned": True},
    client = client
)

- Open the stac catalog

In [11]:
catalog = Client.open("https://explorer.digitalearth.africa/stac")

#### Load Rainfal Data for a farmer in Kisumu County

In [11]:
## Get fields data:
field = gpd.read_file('dr_okello_field.geojson')
field_projected = field.to_crs("EPSG:32736") 

In [29]:
centroid = field.centroid
centroid = centroid.to_crs(4326)
lat = centroid.geometry.y[0]
lng = centroid.geometry.x[0]

/tmp/ipykernel_190258/1728564661.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = field.centroid


In [30]:
centroid.xs

<bound method NDFrame.xs of 0    POINT (34.85532 -0.04392)
dtype: geometry>

In [16]:
rainfall_data = load_rainfall_data_from_chirps(lat, lng, '2023-03-01','2023-07-30')

/home/billy/anaconda3/envs/field_monitoring/lib/python3.10/site-packages/pystac_client/item_search.py:826: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


--- Found:  153 datasets ---


In [19]:
rainfall_ds = rainfall_data.compute()

In [20]:
rainfall_ds

<xarray.Dataset>
Dimensions:      (time: 5, latitude: 0, longitude: 1)
Coordinates:
  * latitude     (latitude) float64 
  * longitude    (longitude) float64 34.88
    spatial_ref  int32 4326
  * time         (time) datetime64[ns] 2023-03-15 2023-04-15 ... 2023-07-15
Data variables:
    rainfall     (time, latitude, longitude) float32

#### Get S2 Data between January to September for field